In [3]:
#!/usr/bin/env python
# coding: utf-8


import argparse
from joblib import Parallel, delayed
import numpy as np
import os
import sys

sys.path.append("/wynton/home/ahituv/fongsl/tools/py_/")
sys.path.append("/wynton/home/ahituv/fongsl/tools/genome/")

import chr_functions
import split_filename

In [4]:
"""
Output: N iterations of shuffled.bed with length-matched shuffled regions, chromosome-matched, non-overlapping, including regions specificed from incl arg

Input: .bed file, n shuffle iterations, genome build, background, sharedAcc, 50bp =< 0000 regions to include as the shuffle

Functions: 
- bedtools shuffle. 

Notes
- length match
- chromosome match
- include = only shuffle regions inside the included file. 


"""

###
#   arguments
###

arg_parser = argparse.ArgumentParser(description= "compute sequence identity between two species")

arg_parser.add_argument("-b","--bedfile", help='.bed file in species 1 coordinates')
arg_parser.add_argument("-i","--iters", type=int, help='number of shuffles')
arg_parser.add_argument("-g","--genome_build", help='genome_build')
arg_parser.add_argument("-incl","--include", help='regions to include in shuffle')
arg_parser.add_argument("-o","--outdir", help='directory for files')

args = arg_parser.parse_args()

TEST_BED = args.bedfile
ITERS = args.iters
BUILD = args.genome_build
INCLUDE = args.include
SHUF_PATH = args.outdir

usage: ipykernel_launcher.py [-h] [-b BEDFILE] [-i ITERS] [-g GENOME_BUILD]
                             [-incl INCLUDE] [-o OUTDIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /wynton/home/ahituv/fongsl/.local/share/jupyter/runtime/kernel-5df59602-739c-4fcf-8181-9d4f6f5e0800.json


SystemExit: 2

In [9]:
TEST_BED = "/wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_overlap.bed"
TEST_BED ="/wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_no-overlap.bed"
#TEST_BED="/wynton/home/ahituv/fongsl/nullomers/data/test.bed"
ITERS = 100
BUILD = "hg38"
INCLUDE = "/wynton/home/ahituv/fongsl/nullomers/data/Gencode/gencode.v42.basic.annotation-exons_merged.bed"
SHUF_PATH = "/wynton/home/ahituv/fongsl/nullomers/data/shuffle"

In [10]:
PATH, FILENAME, SAMPLE_ID = split_filename.split_filename(TEST_BED) 


###
#   functions
###


def loadConstants(build):  
    path_dict = {
                'hg19': ("/wynton/home/ahituv/fongsl/dna/black_list/hg19-blacklist.v2.bed", "/wynton/home/ahituv/fongsl/dna/hg19/hg19.chrom.sizes"),
                'hg38': ("/wynton/home/ahituv/fongsl/dna/black_list/hg38-blacklist.v2.gencode.v42.bed", "/wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes"),


                    }
    blacklist, sizes = path_dict[build]
    
    return blacklist, sizes

In [11]:
def shuffle(test_bed, shuf_path, sample_id, iter_, build, include):
    
    out=os.path.join(shuf_path, f"shuf-{sample_id}-{iter_}.bed")
    
    #out=os.path.join(shuf_path, "test.bed")
    
    BLACKLIST, CHROM_SZ = loadConstants(build)
    if include is not None:
        BEDshuf = f"bedtools shuffle -i {test_bed} -g {CHROM_SZ} -noOverlapping -maxTries 5000 -incl {include} > {out}"


    else:
        BEDshuf = f"bedtools shuffle -i {test_bed} -g {CHROM_SZ} -excl {BLACKLIST} -chrom -noOverlapping -maxTries 5000 > {out}" 

    print(BEDshuf)
    
    os.system(BEDshuf)

In [ ]:
###
#   Main
###


def main(argv):
    
    #num_cores = multiprocessing.cpu_count()
    num_cores = 16
    print("number of cores", num_cores, "number of iters", ITERS)

    # run parallel jobs

    Parallel(n_jobs=num_cores, verbose=100, prefer="threads")(delayed(shuffle)(TEST_BED, SHUF_PATH, SAMPLE_ID, i, BUILD, INCLUDE) for i in range(int(ITERS)))


    #shuffle(TEST_BED, SHUF_PATH, SAMPLE_ID, ITERS, BUILD, INCLUDE)
    
if __name__ == "__main__":
    main(sys.argv[1:])

number of cores 16 number of iters 100
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_no-overlap.bed -g /wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes -noOverlapping -maxTries 5000 -incl /wynton/home/ahituv/fongsl/nullomers/data/Gencode/gencode.v42.basic.annotation-exons_merged.bed > /wynton/home/ahituv/fongsl/nullomers/data/shuffle/shuf-mutations_uniq_lociGENCODE-EXON_no-overlap-0.bed
bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_no-overlap.bed -g /wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes -noOverlapping -maxTries 5000 -incl /wynton/home/ahituv/fongsl/nullomers/data/Gencode/gencode.v42.basic.annotation-exons_merged.bed > /wynton/home/ahituv/fongsl/nullomers/data/shuffle/shuf-mutations_uniq_lociGENCODE-EXON_no-overlap-1.bed
bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutations_u

bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_no-overlap.bed -g /wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes -noOverlapping -maxTries 5000 -incl /wynton/home/ahituv/fongsl/nullomers/data/Gencode/gencode.v42.basic.annotation-exons_merged.bed > /wynton/home/ahituv/fongsl/nullomers/data/shuffle/shuf-mutations_uniq_lociGENCODE-EXON_no-overlap-32.bed
[Parallel(n_jobs=16)]: Done  17 tasks      | elapsed: 94.1min
bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutations_uniq_lociGENCODE-EXON_no-overlap.bed -g /wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes -noOverlapping -maxTries 5000 -incl /wynton/home/ahituv/fongsl/nullomers/data/Gencode/gencode.v42.basic.annotation-exons_merged.bed > /wynton/home/ahituv/fongsl/nullomers/data/shuffle/shuf-mutations_uniq_lociGENCODE-EXON_no-overlap-33.bed[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed: 94.7min

bedtools shuffle -i /wynton/home/ahituv/fongsl/nullomers/data/mutati